# Fake News Detection with NLP

## Initial data reading and preparing

In [21]:
import numpy as np
import pandas as pd

Loading Dataset

In [22]:
train = pd.read_csv('fake_or_real_news_training.csv', index_col="ID", encoding='utf-8')
test = pd.read_csv('fake_or_real_news_test.csv', index_col="ID" , encoding='utf-8')

Taking a look to the data

In [23]:
train.head()

,title,text,label,X1,X2
ID,,,,,
8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,NaN,NaN
10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,NaN,NaN
3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,NaN,NaN
10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,NaN,NaN
875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,NaN,NaN


In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3999 entries, 8476 to 9673
Data columns (total 5 columns):
title    3999 non-null object
text     3999 non-null object
label    3999 non-null object
X1       33 non-null object
X2       2 non-null object
dtypes: object(5)
memory usage: 187.5+ KB


### Joining available data
Let's join title and text fields

In [35]:
df = train.title + str(' ') + train.text
df.head()

ID
8476     You Can Smell Hillary’s Fear Daniel Greenfield...
10294    Watch The Exact Moment Paul Ryan Committed Pol...
3608     Kerry to go to Paris in gesture of sympathy U....
10142    Bernie supporters on Twitter erupt in anger ag...
875      The Battle of New York: Why This Primary Matte...
dtype: object

## Preprocessing and building the term document matrix

### Tokenization and Lemmatization

In [36]:
import nltk
nltk.download()
nltk.download('wordnet')
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/javiergranda/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Term document matrix

Building the term document matrix using sklearn CountVectorizer. Also removing stop words and combining bi-grams to capture word relationships. Also using previously defined tokenizer.

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(tokenizer=LemmaTokenizer(), stop_words='english', ngram_range=(1, 2)) # remove english stop words and applying bi-grams

X_train_counts = count_vect.fit_transform(df) # using the text and title from training set

X_train_counts.shape # take a look at how many featuress where created

(3999, 1002421)

## Removing sparcity with TF-IDF

Using the tf–idf transform to re-weight the count features into floating point values suitable to use with the classifier

In [38]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

## Naive Bayes

Building the classifier

In [40]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train_tfidf, train.label)

### Cross-validation of the Naive Bayes classifier

In [41]:
import warnings; warnings.simplefilter('ignore')
from sklearn.model_selection import cross_val_score

cv_score_nb = cross_val_score(
    clf,
    X_train_tfidf,
    train.label,
    cv = 5,
    n_jobs = -1)

print('Train - Naive Bayes cross validated score is: '+ str(np.mean(cv_score_nb)))

Train - Naive Bayes cross validated score is: 0.7975685067801546


# ****** THEO'S MAX ENTROPY CLASSIFIER ********** ######

In [45]:
### CODE HERE......

## SVM on regularized linear models with stochastic gradient descent (SGD) learning

In [46]:
from sklearn.linear_model import SGDClassifier

svm_clf = SGDClassifier(
    loss='perceptron', 
    penalty='l2',
    alpha=1e-3,
    max_iter=5,
    random_state=42).fit(X_train_tfidf, train.label)

In [47]:
import warnings; warnings.simplefilter('ignore')

cv_score_svm = cross_val_score(
    svm_clf,
    X_train_tfidf,
    train.label,
    cv = 5,
    n_jobs = -1)

print('Train - SVM cross validated score is: '+ str(np.mean(cv_score_svm)))

Train - SVM cross validated score is: 0.930557126977132


In [276]:
# FIX GRADIENT SEARCH
from sklearn.model_selection import GridSearchCV
params_svm = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf-svm__alpha': (1e-2, 1e-3),}

gs_clf_svm = GridSearchCV(svm_clf, params_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X_train_tfidf, train.label)
gs_clf_svm.best_score_
gs_clf_svm.best_params_

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/javiergranda/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Users/javiergranda/an